In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import re

In [ ]:
!pip install pythainlp
from pythainlp.util import normalize
from pythainlp import correct
!pip install sentencepiece
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 7.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.3 MB/s 


In [ ]:
from pythainlp import word_tokenize, Tokenizer
from pythainlp.corpus.common import thai_words
from pythainlp.util import Trie

# Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab_Notebooks/IS/Test_IS

/content/drive/MyDrive/Colab_Notebooks/IS/Test_IS


In [ ]:
colnames=['text', 'MASK_NUM', 'PRON', 'REF'] 
df = pd.read_csv('zero_list.txt',names=colnames, header=None)

In [ ]:
df

,text,MASK_NUM,PRON,REF
0,ผู้ชายที่กำลังพูดโทรศัพท์<mask>ไม่ค่อยชอบฉัน,<mask_3rd>,เขา,ผู้ชาย
1,ผู้ชายที่ฉันชอบ<mask>กำลังพูดโทรศัพท์,<mask_3rd>,เขา,ผู้ชาย
2,พนักงานที่เธอเห็นเมื่อเช้า<mask>ขอลากลับก่อนนะ,<mask_3rd>,เขา,พนักงาน
3,เธอเห็นแมวตัวที่พ่อกำลังอุ้ม<mask>อยู่ตรงนั้นไหม,<mask_3rd>,มัน,แมว
4,วันนี้ฉันเจอลิงที่เธอเจอ<mask>เมื่อวาน,<mask_3rd>,มัน,ลิง
...,...,...,...,...
295,กีต้าร์ของพี่ที่น้องชอบยังใหม่อยู่เลยนะ พี่ขาย...,<mask_3rd>,มัน,กีต้าร์
296,<mask>ฟังเพลงนี้ทีไร กูนึกถึงแฟนเก่าทุกทีเลย,<mask_1st>,กู,กู
297,วันจันทร์มึงหยุดมะ กุว่าจะชวน<mask>ไปดูคอนโด,<mask_2nd>,มึง,มึง
298,สรุป<mask>จะไปตีกอล์ฟวันไหนนะครับพี่ ผมลืมสนิทเลย,<mask_2nd>,พี่,พี่


In [ ]:
df.shape

(300, 4)

In [ ]:
df['MASK_NUM'].value_counts()

<mask_3rd>    116
<mask_2nd>     93
<mask_1st>     91
Name: MASK_NUM, dtype: int64

#Preprocessing Data

## Data cleansing

In [ ]:
df = df.applymap(str)

In [ ]:
def clean_text(text:str):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  text = str(text).strip()
  text = normalize(text)
  text = re.sub(emoji_pattern,r'', text) #clean emoji
  text = re.sub(r'(\D)(\1{2,})',r'\1', text)
  text = re.sub(r'[“”„]', '"', text) # convert double quotations into "
  text = re.sub(r'[‘’′′′′`]', "'", text) # convert single quotations into '
  text = re.sub(r'[ \n\t\u00a0\xa0\u3000\u2002-\u200a\u0020+]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
  text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
  text = re.sub(r'ํา','ำ', text) # o + า -> ำ
  text = re.sub(r'\u0E33([\u0E48\u0E49\u0E4A\u0E4B])', r'\1'+'\u0E33', text) # am + tone -> tone + am
  text = re.sub(r'[<>]','',text)
  
  return text


In [ ]:
df['text'] = df['text'].apply(clean_text)

In [ ]:
print(df.loc[6])

text        สุดานะ ฉันได้ยินข่าวว่าmaskเพิ่งพาสมศรีไปโรงพย...
MASK_NUM                                           <mask_3rd>
PRON                                                      เขา
REF                                                      สุดา
Name: 6, dtype: object


In [ ]:
df

,text,MASK_NUM,PRON,REF
0,ผู้ชายที่กำลังพูดโทรศัพท์maskไม่ค่อยชอบฉัน,<mask_3rd>,เขา,ผู้ชาย
1,ผู้ชายที่ฉันชอบmaskกำลังพูดโทรศัพท์,<mask_3rd>,เขา,ผู้ชาย
2,พนักงานที่เธอเห็นเมื่อเช้าmaskขอลากลับก่อนนะ,<mask_3rd>,เขา,พนักงาน
3,เธอเห็นแมวตัวที่พ่อกำลังอุ้มmaskอยู่ตรงนั้นไหม,<mask_3rd>,มัน,แมว
4,วันนี้ฉันเจอลิงที่เธอเจอmaskเมื่อวาน,<mask_3rd>,มัน,ลิง
...,...,...,...,...
295,กีต้าร์ของพี่ที่น้องชอบยังใหม่อยู่เลยนะ พี่ขาย...,<mask_3rd>,มัน,กีต้าร์
296,maskฟังเพลงนี้ทีไร กูนึกถึงแฟนเก่าทุกทีเลย,<mask_1st>,กู,กู
297,วันจันทร์มึงหยุดมะ กุว่าจะชวนmaskไปดูคอนโด,<mask_2nd>,มึง,มึง
298,สรุปmaskจะไปตีกอล์ฟวันไหนนะครับพี่ ผมลืมสนิทเลย,<mask_2nd>,พี่,พี่


In [ ]:
### normalize non-standard สรรพนาม to standard words ###
#เค้า --> เขา regex
#เทอ --> เธอ regex
#เตง --> ตัวเอง regex
#เมิง --> มึง regex

#กุ --> กู token
#เลา --> เรา token

#x แกร --> แก (แก+ร/แกร)
#x ชั้น --> ฉัน

#ใช้ token
dict_correct_prons = {'เค้า':'เขา', 'เทอ':'เธอ', 'เตง':'ตัวเอง', 'เมิง':'มึง', 'กุ':'กู', 'เลา':'เรา', 'แกร':'แก', 'ชั้น':'ฉัน', 'กูญแจ':'กุญแจ', 'ฉันสูง':'ชั้นสูง'}

In [ ]:
def correct_pron(text):
  text = str(text).strip()
  for pron in dict_correct_prons.keys():
    text = re.sub(pron, dict_correct_prons[pron], text)
  return text

correct_pron('เตงจะไปหาเค้าเมื่อไหร่ กุญแจล่ะ เทอๆ ชั้นสูงป่ะ')

'ตัวเองจะไปหาเขาเมื่อไหร่ กุญแจล่ะ เธอๆ ชั้นสูงป่ะ'

In [ ]:
df['text'] = df.text.apply(correct_pron)

In [ ]:
df

,text,MASK_NUM,PRON,REF
0,ผู้ชายที่กำลังพูดโทรศัพท์maskไม่ค่อยชอบฉัน,<mask_3rd>,เขา,ผู้ชาย
1,ผู้ชายที่ฉันชอบmaskกำลังพูดโทรศัพท์,<mask_3rd>,เขา,ผู้ชาย
2,พนักงานที่เธอเห็นเมื่อเช้าmaskขอลากลับก่อนนะ,<mask_3rd>,เขา,พนักงาน
3,เธอเห็นแมวตัวที่พ่อกำลังอุ้มmaskอยู่ตรงนั้นไหม,<mask_3rd>,มัน,แมว
4,วันนี้ฉันเจอลิงที่เธอเจอmaskเมื่อวาน,<mask_3rd>,มัน,ลิง
...,...,...,...,...
295,กีต้าร์ของพี่ที่น้องชอบยังใหม่อยู่เลยนะ พี่ขาย...,<mask_3rd>,มัน,กีต้าร์
296,maskฟังเพลงนี้ทีไร กูนึกถึงแฟนเก่าทุกทีเลย,<mask_1st>,กู,กู
297,วันจันทร์มึงหยุดมะ กูว่าจะชวนmaskไปดูคอนโด,<mask_2nd>,มึง,มึง
298,สรุปmaskจะไปตีกอล์ฟวันไหนนะครับพี่ ผมลืมสนิทเลย,<mask_2nd>,พี่,พี่


## Tokenize

In [ ]:
#custom_word = ['mask']
#custom_tokenizer = Tokenizer(custom_word)

In [ ]:
new_words = {"mask"}
words = new_words.union(thai_words())

In [ ]:
custom_dictionary_trie = Trie(words)
df['tokens'] = df['text'].apply(lambda x: word_tokenize(x, custom_dict=custom_dictionary_trie))
#df['token'] = df['text'].apply(lambda x: custom_tokenizer.word_tokenize(x))
#df['token'] = df['text'].apply(lambda x: word_tokenize(x, engine='attacut',keep_whitespace=True))

In [ ]:
df.applymap(type).apply(pd.value_counts).fillna(0)

,text,MASK_NUM,PRON,REF,tokens
<class 'str'>,300.0,300.0,300.0,300.0,0.0
<class 'list'>,0.0,0.0,0.0,0.0,300.0


## Annotate tokens

In [ ]:
def annotate_token(data_frame,token_colname='tokens',label_colname='MASK_NUM',new_colname='num_label'):
  list_anno = []
  for i,row in data_frame.iterrows(): #for loop row each row
    list_tokens = []
    for token in row[4]: #for each token in tokens
      if token != 'mask':
        list_tokens.append('N')
      else:
        if row[1] == '<mask_3rd>':
          list_tokens.append('mask_3rd')
        elif row[1] == '<mask_2nd>':
          list_tokens.append('mask_2nd')
        elif row[1] == '<mask_1st>':
          list_tokens.append('mask_1st')
      
    list_anno.append(list_tokens)
  data_frame[new_colname] = list_anno
  df['n_tokens'] = df['tokens'].apply(len)
  return data_frame

In [ ]:
df2 = df.replace({'Courses': 'Py', 'Duration': 'days'}, 
    {'Courses': 'Python with', 'Duration': ' Days'}, regex=True)

In [ ]:
annotate_token(df)

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens
0,ผู้ชายที่กำลังพูดโทรศัพท์maskไม่ค่อยชอบฉัน,<mask_3rd>,เขา,ผู้ชาย,"[ผู้ชาย, ที่, กำลัง, พูดโทรศัพท์, mask, ไม่ค่อ...","[N, N, N, N, mask_3rd, N, N, N]",8
1,ผู้ชายที่ฉันชอบmaskกำลังพูดโทรศัพท์,<mask_3rd>,เขา,ผู้ชาย,"[ผู้ชาย, ที่, ฉัน, ชอบ, mask, กำลัง, พูดโทรศัพท์]","[N, N, N, N, mask_3rd, N, N]",7
2,พนักงานที่เธอเห็นเมื่อเช้าmaskขอลากลับก่อนนะ,<mask_3rd>,เขา,พนักงาน,"[พนักงาน, ที่, เธอ, เห็น, เมื่อเช้า, mask, ขอล...","[N, N, N, N, N, mask_3rd, N, N, N, N]",10
3,เธอเห็นแมวตัวที่พ่อกำลังอุ้มmaskอยู่ตรงนั้นไหม,<mask_3rd>,มัน,แมว,"[เธอ, เห็น, แมว, ตัว, ที่, พ่อ, กำลัง, อุ้ม, m...","[N, N, N, N, N, N, N, N, mask_3rd, N, N, N]",12
4,วันนี้ฉันเจอลิงที่เธอเจอmaskเมื่อวาน,<mask_3rd>,มัน,ลิง,"[วันนี้, ฉัน, เจอ, ลิง, ที่, เธอ, เจอ, mask, เ...","[N, N, N, N, N, N, N, mask_3rd, N]",9
...,...,...,...,...,...,...,...
295,กีต้าร์ของพี่ที่น้องชอบยังใหม่อยู่เลยนะ พี่ขาย...,<mask_3rd>,มัน,กีต้าร์,"[กีต้าร์, ของ, พี่, ที่, น้อง, ชอบ, ยัง, ใหม่,...","[N, N, N, N, N, N, N, N, N, N, N, N, N, mask_3...",17
296,maskฟังเพลงนี้ทีไร กูนึกถึงแฟนเก่าทุกทีเลย,<mask_1st>,กู,กู,"[mask, ฟังเพลง, นี้, ทีไร, , กู, นึกถึง, แฟน,...","[mask_1st, N, N, N, N, N, N, N, N, N, N]",11
297,วันจันทร์มึงหยุดมะ กูว่าจะชวนmaskไปดูคอนโด,<mask_2nd>,มึง,มึง,"[วัน, จันทร์, มึง, หยุด, มะ, , กู, ว่า, จะ, ช...","[N, N, N, N, N, N, N, N, N, N, mask_2nd, N, N]",13
298,สรุปmaskจะไปตีกอล์ฟวันไหนนะครับพี่ ผมลืมสนิทเลย,<mask_2nd>,พี่,พี่,"[สรุป, mask, จะ, ไป, ตี, กอล์ฟ, วัน, ไหน, นะ, ...","[N, mask_2nd, N, N, N, N, N, N, N, N, N, N, N,...",15


In [ ]:
#df.n_tokens.value_counts(ascending=False)

In [ ]:
df['tokens'] = df['tokens'].apply(lambda x: '|'.join(x))
df['num_label'] = df['num_label'].apply(lambda x: '|'.join(x))

# Split train, dev, test

In [ ]:
seed = 45

train, dev_test = train_test_split(df, train_size=0.8, random_state=seed)
dev, test = train_test_split(dev_test, test_size=0.5, random_state=seed)
train_df = train.reset_index(drop=True)
dev_df = dev.reset_index(drop=True)
test_df = test.reset_index(drop=True)

In [ ]:
train_df['MASK_NUM'].value_counts()

<mask_3rd>    94
<mask_1st>    74
<mask_2nd>    72
Name: MASK_NUM, dtype: int64

In [ ]:
train_df.head()

,text,MASK_NUM,PRON,REF,tokens,num_label,n_tokens
0,ปีเตอร์นะ ฉันได้ยินข่าวว่าจอห์นเพิ่งพาmaskไปโร...,<mask_3rd>,เขา,ปีเตอร์,ปีเตอร์|นะ| |ฉัน|ได้ยิน|ข่าว|ว่า|จอห์น|เพิ่ง|พ...,N|N|N|N|N|N|N|N|N|N|mask_3rd|N|N|N,14
1,ทำไมส่วนใหญ่ญาติต้องอิจฉากันเอง ทำไมmaskถึงไม่...,<mask_3rd>,พวกเขา,ญาติ,ทำไม|ส่วนใหญ่|ญาติ|ต้อง|อิจฉา|กันเอง| |ทำไม|ma...,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13
2,เมื่อก่อนตอนเธออ้วน maskดูแก่กว่าปัจจุบันอีก,<mask_2nd>,เธอ,เธอ,เมื่อก่อน|ตอน|เธอ|อ้วน| |mask|ดู|แก่|กว่า|ปัจจ...,N|N|N|N|N|mask_2nd|N|N|N|N|N,11
3,ไปถามพี่เขาหน่อยซิว่าmaskอยากจะกินข้าวที่ไหน,<mask_3rd>,เขา,พี่,ไป|ถาม|พี่|เขา|หน่อย|ซิ|ว่า|mask|อยาก|จะ|กินข้...,N|N|N|N|N|N|N|mask_3rd|N|N|N|N,12
4,เฮียแกกลับมาร้องเพลงได้แล้ว สงสัยmaskเลิกยาถาว...,<mask_3rd>,เขา,เฮีย,เฮีย|แก|กลับมา|ร้องเพลง|ได้|แล้ว| |สงสัย|mask|...,N|N|N|N|N|N|N|N|mask_3rd|N|N|N|N,13


In [ ]:
train_df.to_csv('train_df_normalized.csv',encoding='utf-8-sig', index=False,)
dev_df.to_csv('dev_df_normalized.csv',encoding='utf-8-sig', index=False,)
test_df.to_csv('test_df_normalized.csv',encoding='utf-8-sig', index=False,)